In [10]:
import psycopg2
import json
import string
with open("issues_final.json", 'r') as final_json_file:
    data = json.load(final_json_file)

In [ ]:
host = "ec2-99-80-170-190.eu-west-1.compute.amazonaws.com"
database = "d7t29ibovqda27"
user = "yerpxinllkwyth"
password = '0de087f43b0947fc0c4ee384a724d1bcb39f5dbda2e63c972c6ac7e33805f1a2'
port = 5432
conn = None
cur = None
try:
    conn = psycopg2.connect(
            host = host,
            user = user,
            dbname = database,
            password = password,
            port = port)

    cur = conn.cursor()
    SQLscript = '''
                INSERT INTO issues (id, title, body) VALUES (%s, %s, %s)
                '''
    for current in data:
        id = current["originalId"]
        title = ' '.join(current["originalTitle"].translate(str.maketrans(string.punctuation, ' '*len(string.punctuation))).lower().split())
        body = ' '.join(current["originalBody"].translate(str.maketrans(string.punctuation, ' '*len(string.punctuation))).lower().split())
        values = (id, title, body)
        try:    
            cur.execute(SQLscript, values)
            conn.commit()
        except Exception as SQLerr:
            conn.rollback()
            print(SQLerr)

except Exception as error:
    print(error)

In [20]:
if cur is not None:
    cur.close()
if conn is not None:
    conn.close()